In [1]:
import numpy as np
import pandas as pd
from river import optim
from sail.models.river.forest import AdaptiveRandomForestClassifier
from sail.pipeline import SAILPipeline
from sklearn.impute import SimpleImputer
from sail.transfomers.river.preprocessing import StandardScaler

#### Load Data


In [2]:
X = pd.read_csv("../datasets/agrawal.csv").head(5000)

y = X["class"]
X.drop("class", axis=1, inplace=True)

#### Model Definition


In [3]:
random_forest = AdaptiveRandomForestClassifier(n_models=10)

#### Create SAIL Pipeline


In [4]:
steps = [
    ("Imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("standard_scalar", StandardScaler()),
    ("classifier", random_forest),
]
sail_pipeline = SAILPipeline(steps=steps)

#### Train Test Split


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

#### Start Incremental Training


In [6]:
y_preds = []
y_true = []
batch_size = 100

for start in range(0, X_train.shape[0], batch_size):

    end = start + batch_size

    X = X_train.iloc[start:end]
    y = y_train.iloc[start:end]


    sail_pipeline.partial_fit(X, y, classifier__classes=[1, 0])

SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Total:0.088s, Batch Size=100, Score=0]                  
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Total:0.076s, Batch Size=100, Score=0.68]                  
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Total:0.077s, Batch Size=100, Score=0.705]                  
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Total:0.080s, Batch Size=100, Score=0.783]                  
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Total:0.081s, Batch Size=100, Score=0.83]                  
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Total:0.079s, Batch Size=100, Score=0.844]                  
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Total:0.079s, Batch Size=100, Score=0.857]                  
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00

#### Save SAIL Pipeline


In [7]:
sail_pipeline.save(".")

[2023-06-30 00:14:30:614] - INFO - SAIL : making directory tree ./sail_pipeline


'./sail_pipeline'

#### Load SAIL Pipeline


In [8]:
sail_new_pipeline = SAILPipeline.load(".")

#### Make Prediction on the Hold out set


In [9]:
y_preds = []
y_true = []
batch_size = 100

for start in range(0, X_test.shape[0], batch_size):

    end = start + batch_size

    X = X_test.iloc[start:end]
    y = y_test.iloc[start:end]

    preds = sail_new_pipeline.predict(X)
    y_preds.extend(list(preds))
    y_true.extend(list(y))

#### Classification Report


In [10]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_true, y_preds))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       992
           1       0.92      0.89      0.91       508

    accuracy                           0.94      1500
   macro avg       0.93      0.93      0.93      1500
weighted avg       0.94      0.94      0.94      1500



#### Plot confusion matrix


<Axes: >

In [11]:
import seaborn as sns

cf_matrix = confusion_matrix(y_true, y_preds)
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, fmt='.2%', cmap='Blues')